## Underfitting

In [1]:
import tensorflow as tf
import numpy as np

In [7]:
EPOCHS = 100

## Model Define

In [3]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(32, activation='relu')
        self.dense2 = tf.keras.layers.Dense(64, activation='relu')
        self.dense3 = tf.keras.layers.Dense(128, activation='relu')
        self.dense4 = tf.keras.layers.Dense(256, activation='relu')
        self.dense5 = tf.keras.layers.Dense(10, activation='softmax')
        
    def call(self, x, training=False, mask=None):
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.dense4(x)
        return self.dense5(x)

## Dataset : Fashion MNIST

In [4]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = x_train[..., tf.newaxis].astype(np.float32)
x_test = x_test[..., tf.newaxis].astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32).prefetch(2048)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32).prefetch(2048)

4423680/4422102 [==============================] - 0s 0us/step


## Training with Keras API

In [8]:
model = MyModel()
model.compile(optimizer='adam',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])
model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS);

Train for 1875 steps, validate for 313 steps
Epoch 1/100
1875/1875 [==============================] - 4s 2ms/step - loss: 0.5194 - accuracy: 0.8100 - val_loss: 0.4327 - val_accuracy: 0.8411
Epoch 2/100
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3866 - accuracy: 0.8560 - val_loss: 0.4322 - val_accuracy: 0.8343
Epoch 3/100
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3516 - accuracy: 0.8707 - val_loss: 0.3927 - val_accuracy: 0.8576
Epoch 4/100
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3295 - accuracy: 0.8774 - val_loss: 0.3841 - val_accuracy: 0.8596
Epoch 5/100
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3155 - accuracy: 0.8832 - val_loss: 0.3625 - val_accuracy: 0.8688
Epoch 6/100
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3009 - accuracy: 0.8870 - val_loss: 0.3518 - val_accuracy: 0.8722
Epoch 7/100
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2915 -

Epoch 57/100
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1368 - accuracy: 0.9473 - val_loss: 0.5866 - val_accuracy: 0.8803
Epoch 58/100
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1435 - accuracy: 0.9451 - val_loss: 0.5938 - val_accuracy: 0.8800
Epoch 59/100
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1315 - accuracy: 0.9488 - val_loss: 0.6101 - val_accuracy: 0.8772
Epoch 60/100
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1382 - accuracy: 0.9468 - val_loss: 0.7430 - val_accuracy: 0.8768
Epoch 61/100
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1393 - accuracy: 0.9485 - val_loss: 0.5725 - val_accuracy: 0.8816
Epoch 62/100
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1316 - accuracy: 0.9491 - val_loss: 0.7349 - val_accuracy: 0.8711
Epoch 63/100
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1311 - accuracy: 0.9503 - val_loss: 0.6486 -

## Architecture improvement with DenseNet

In [9]:
class MyModelD(tf.keras.Model):
    def __init__(self):
        super(MyModelD, self).__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(32, use_bias=False)
        self.batch1 = tf.keras.layers.BatchNormalization()
        
        self.batch2 = tf.keras.layers.BatchNormalization()
        self.dense2 = tf.keras.layers.Dense(32, use_bias=False)
        
        self.batch3 = tf.keras.layers.BatchNormalization()
        self.dense3 = tf.keras.layers.Dense(64, use_bias=False)
        
        self.batch4 = tf.keras.layers.BatchNormalization()
        self.dense4 = tf.keras.layers.Dense(128, use_bias=False)
                
        self.dense5 = tf.keras.layers.Dense(10, activation='softmax')
        
    def call(self, x, training=False, mask=None):
        x = self.flatten(x)
        
        x = self.dense1(x)
        x = self.batch1(x, training)
        x = tf.nn.relu(x)
        
        h = self.batch2(x, training)
        h = tf.nn.relu(h)
        h = self.dense2(h)
        x = tf.concat([x, h], axis=-1)
        
        h = self.batch3(x, training)
        h = tf.nn.relu(h)
        h = self.dense3(h)
        x = tf.concat([x, h], axis=-1)
        
        h = self.batch4(x, training)
        h = tf.nn.relu(h)
        h = self.dense4(h)
        x = tf.concat([x, h], axis=-1)
        
        return self.dense5(x)

In [10]:
model = MyModelD()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])
model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS);

Train for 1875 steps, validate for 313 steps
Epoch 1/100
1875/1875 [==============================] - 7s 4ms/step - loss: 0.5104 - accuracy: 0.8151 - val_loss: 0.4980 - val_accuracy: 0.8138
Epoch 2/100
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3974 - accuracy: 0.8558 - val_loss: 0.5053 - val_accuracy: 0.8367
Epoch 3/100
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3679 - accuracy: 0.8656 - val_loss: 0.4249 - val_accuracy: 0.8456
Epoch 4/100
1875/1875 [==============================] - 7s 3ms/step - loss: 0.3484 - accuracy: 0.8723 - val_loss: 0.3873 - val_accuracy: 0.8609
Epoch 5/100
1875/1875 [==============================] - 7s 3ms/step - loss: 0.3338 - accuracy: 0.8773 - val_loss: 0.4374 - val_accuracy: 0.8406
Epoch 6/100
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3211 - accuracy: 0.8818 - val_loss: 0.3589 - val_accuracy: 0.8717
Epoch 7/100
1875/1875 [==============================] - 7s 3ms/step - loss: 0.3113 -

Epoch 57/100
1875/1875 [==============================] - 7s 3ms/step - loss: 0.1889 - accuracy: 0.9286 - val_loss: 0.4346 - val_accuracy: 0.8716
Epoch 58/100
1875/1875 [==============================] - 7s 3ms/step - loss: 0.1866 - accuracy: 0.9296 - val_loss: 0.4382 - val_accuracy: 0.8734
Epoch 59/100
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1880 - accuracy: 0.9291 - val_loss: 0.4401 - val_accuracy: 0.8756
Epoch 60/100
1875/1875 [==============================] - 7s 3ms/step - loss: 0.1832 - accuracy: 0.9311 - val_loss: 0.4137 - val_accuracy: 0.8780
Epoch 61/100
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1845 - accuracy: 0.9300 - val_loss: 0.4526 - val_accuracy: 0.8722
Epoch 62/100
1875/1875 [==============================] - 7s 3ms/step - loss: 0.1832 - accuracy: 0.9305 - val_loss: 0.4051 - val_accuracy: 0.8816
Epoch 63/100
1875/1875 [==============================] - 7s 3ms/step - loss: 0.1838 - accuracy: 0.9301 - val_loss: 0.4013 -